In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV 
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle
from mlxtend.regressor import StackingRegressor

from scipy.special import erfc

In [ ]:
df = pd.read_csv('final_all_editted.csv')
df = shuffle(df) # 좀 더 랜덤한 학습을 위해 데이터를 섞어줌

FileNotFoundError: ignored

In [ ]:
df['y'] = df['최근매매실거래가격'] / df['아파트면적값']
# y에 로그 씌움
df.y = np.log(df.y)

train = df.loc[(df.기준년월==201904)|(df.기준년월==201905),
               ['아파트세대수', '아파트동수', '아파트면적값', '서울특별시', '강원도', '대구특별시', '면적', '인구', '세대', '대형마트수_8km이내',
                '지하철수_2km이내', '학교수_1km이내', '공원수_2.6km이내', '인구밀도', '세대당주차대수', '아파트준공일자',
                'CCTV수','부대시설갯수', '승강기수', '관리사무소', '노인정', '문고', '보육시설', '어린이놀이터', '유치원',
                '자전거보관소', '주민공동시설', '커뮤니티공간', '휴게시설', '주택거래건수', 
                'mean.주가수익비율', 'mean.주가현금흐름비율', 'moving_num','y']]
valid = df.loc[df.기준년월==202004, ['아파트세대수', '아파트동수', '아파트면적값', '서울특별시', '강원도', '대구특별시', '면적', '인구', '세대', '대형마트수_8km이내',
                                 '지하철수_2km이내', '학교수_1km이내', '공원수_2.6km이내', '인구밀도', '세대당주차대수', '아파트준공일자',
                                 'CCTV수','부대시설갯수', '승강기수', '관리사무소', '노인정', '문고', '보육시설', '어린이놀이터', '유치원',
                                 '자전거보관소', '주민공동시설', '커뮤니티공간', '휴게시설', '주택거래건수', 
                                 'mean.주가수익비율', 'mean.주가현금흐름비율', 'moving_num','y']]
test = df.loc[df.기준년월==202005, ['아파트세대수', '아파트동수', '아파트면적값', '서울특별시', '강원도', '대구특별시', '면적', '인구', '세대', '대형마트수_8km이내',
                                 '지하철수_2km이내', '학교수_1km이내', '공원수_2.6km이내', '인구밀도', '세대당주차대수', '아파트준공일자',
                                 'CCTV수','부대시설갯수', '승강기수', '관리사무소', '노인정', '문고', '보육시설', '어린이놀이터', '유치원',
                                 '자전거보관소', '주민공동시설', '커뮤니티공간', '휴게시설', '주택거래건수', 
                                 'mean.주가수익비율', 'mean.주가현금흐름비율', 'moving_num','y']]

In [ ]:
def get_xy(data) :
    x = data.drop('y', axis=1)
    y = data.y
    return x,y

train_x, train_y = get_xy(train)
valid_x, valid_y = get_xy(valid)
test_x, test_y = get_xy(test)

# deep learning fitting

- 통계적 가정을 만족시키기 위해 y에 로그를 씌워서 학습시킴
- n이 상대적으로 적다고 판단하여, 1) 미니 배치를 사용하지 않고 2) gradient descent optimizer 사용 3) hidden layer 1개만 사용
- learning rate는 0.001,0.01,0.05 중 가장 안정적인 예측을 하는 0.01로 선택


- 처음엔 relu activation function을 이용했으나, 노드가 죽는 경우 발생 -> 학습이 되지 않음  
- 해결책 1 : batch normalization
- 해결책 2 : activation function을 selu로 변경 (selu activation fn + sequential api가 성능이 좋다고 알려져있음)  

In [ ]:
# selu function

alpha_0_1 = -np.sqrt(2/np.pi) / ( erfc(1/np.sqrt(2)) * np.exp(1/2) -1 )
scale_0_1 = ( 1 - erfc(1/np.sqrt(2)) * np.sqrt(np.e)) * np.sqrt(2*np.pi) * ( 2 * erfc(np.sqrt(2))*np.e**2 + 
                                                                           np.pi*erfc(1/np.sqrt(2))**2*np.e -
                                                                           2*(2+np.pi)*erfc(1/np.sqrt(2))*np.sqrt(np.e) +
                                                                           np.pi + 2 )**(-1/2)

def selu(z, scale = scale_0_1, alpha = alpha_0_1) :
    return scale * tf.where(z>=0.0, z, alpha*tf.nn.elu(z))

In [ ]:
def build_model() :
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(30, activation=selu, input_shape=(train_x.shape[1],)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(15, activation=selu))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(1))
    opt = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=0.01)
    model.compile(optimizer=opt, loss='mse', metrics=['mae'])
    return model

train_y = np.log(train_y)

model = build_model()
result = model.fit(train_x, train_y, epochs=100, validation_split=0.33, batch_size=1, verbose=0)

In [ ]:
## overfitting 확인

plt.plot(result.history['loss'])
plt.plot(result.history['val_loss'])
plt.title('model MSE')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','valid'], loc='upper left')
plt.show()

In [ ]:
## 모델 정교화

from keras import regularizers

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)

def build_model() :
    model = tf.keras.models.Sequential()
    
    ## input layer
    model.add(tf.keras.layers.Dense(30, activation=selu, input_shape=(train_x.shape[1],)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.5))
    
    ## hidden layer
    model.add(tf.keras.layers.Dense(15, activation=selu))
    model.add(tf.keras.layers.BatchNormalization())
              
    ## output layer
    model.add(tf.keras.layers.Dense(1))
    
    ## model compile
    opt = tf.keras.optimizers.Adam(learning_rate=0.005)
    model.compile(optimizer=opt, loss='mse', metrics=['mae'])
    return model

train_y = np.log(train.y)

model = build_model()
result = model.fit(train_x, train_y, epochs=1000, validation_split=0.5, batch_size=1, callbacks=[callback], verbose=0)

In [ ]:
prediction = pd.DataFrame(np.exp(model.predict(valid_x)))
prediction.columns = ['pred']
prediction['true'] = valid.y.reset_index(drop=True)

hyper parameter를 다양하게 조절하여 적합시켜보았으나, 성능이 향상되지 않았음 + overfitting

해당 데이터는 deep learning 모델엔 부적절하다고 판단하여, machine learning 모델을 적용시켜보기로 함

# 다양한 machine learning - regression 모델 적합시켜보기

- 별도의 hyper parameter 없이 validation set R square를 통해 대략적인 모델의 적절성만 파악  
- 이후 괜찮은 모델 몇개만 골라서 hyper parameter tuning 진행

## **라쏘 회귀**

In [ ]:
lasso = Lasso()
lasso.fit(train_x, train_y)

print("train set R^2 : {:.3f}".format(lasso.score(train_x, train_y)))
print("valid set R^2 : {:.3f}".format(lasso.score(valid_x, valid_y)))

train set R^2 : 0.607
valid set R^2 : 0.196


## **릿지 회귀**

In [ ]:
rid = Ridge()
rid.fit(train_x, train_y)

print("train set R^2 : {:.3f}".format(rid.score(train_x, train_y)))
print("valid set R^2 : {:.3f}".format(rid.score(valid_x, valid_y)))

train set R^2 : 0.724
valid set R^2 : 0.478


## **엘라스틱넷 회귀**

In [ ]:
ela = ElasticNet()
ela.fit(train_x, train_y)

print("train set R^2 : {:.3f}".format(ela.score(train_x, train_y)))
print("valid set R^2 : {:.3f}".format(ela.score(valid_x, valid_y)))

train set R^2 : 0.655
valid set R^2 : 0.264


## **의사결정나무 회귀**

In [ ]:
tree = DecisionTreeRegressor(random_state=0)
tree.fit(train_x, train_y)

print("train set R^2 : {:.3f}".format(tree.score(train_x, train_y)))
print("valid set R^2 : {:.3f}".format(tree.score(valid_x, valid_y)))

train set R^2 : 1.000
valid set R^2 : 0.014


## **아다부스트회귀 + 의사결정나무 회귀**

In [ ]:
adatree = AdaBoostRegressor(DecisionTreeRegressor(),random_state=0)
adatree.fit(train_x, train_y)

print("train set R^2 : {:.3f}".format(adatree.score(train_x, train_y)))
print("valid set R^2 : {:.3f}".format(adatree.score(valid_x, valid_y)))

train set R^2 : 1.000
valid set R^2 : 0.408


## **배깅 회귀 - 의사결정나무 기반**

In [ ]:
bb

In [ ]:
bag = BaggingRegressor(random_state=0)
bag.fit(train_x, train_y)

print("train set R^2 : {:.3f}".format(bag.score(train_x, train_y)))
print("valid set R^2 : {:.3f}".format(bag.score(valid_x, valid_y)))

train set R^2 : 0.999
valid set R^2 : 0.155


## **랜덤포레스트 회귀**

In [ ]:
rf = RandomForestRegressor(random_state=0)
rf.fit(train_x, train_y)

print("train set R^2 : {:.3f}".format(rf.score(train_x, train_y)))
print("valid set R^2 : {:.3f}".format(rf.score(valid_x, valid_y)))

train set R^2 : 0.999
valid set R^2 : 0.184


## **그래디언트 부스팅 회귀**

In [ ]:
gbrt = GradientBoostingRegressor(random_state=0)
gbrt.fit(train_x, train_y)

print("train set R^2 : {:.3f}".format(gbrt.score(train_x, train_y)))
print("valid set R^2 : {:.3f}".format(gbrt.score(valid_x, valid_y)))

train set R^2 : 0.919
valid set R^2 : 0.758


## **k neighbors 회귀**

In [ ]:
kn = KNeighborsRegressor()
kn.fit(train_x, train_y)

print("train set R^2 : {:.3f}".format(kn.score(train_x, train_y)))
print("valid set R^2 : {:.3f}".format(kn.score(valid_x, valid_y)))

train set R^2 : 0.995
valid set R^2 : 0.936


# 성능 높은 모델 hyper parameter tuning

## **그래디언트 부스팅 회귀**

In [ ]:
param_gbrt={'n_estimators':[100,200,300,400,500], 
            'learning_rate': [0.1,0.05,0.01],
            'max_depth':[2,5,8,10],
            'max_features':[5,10,15,20,25,30]} 

search_gbrt = RandomizedSearchCV(estimator=GradientBoostingRegressor(random_state=0), cv=5, param_distributions=param_gbrt, n_jobs=-1, n_iter=30)
search_gbrt.fit(train_x, train_y)

RandomizedSearchCV(cv=5, estimator=GradientBoostingRegressor(random_state=0),
                   n_iter=30, n_jobs=-1,
                   param_distributions={'learning_rate': [0.1, 0.05, 0.01],
                                        'max_depth': [2, 5, 8, 10],
                                        'max_features': [5, 10, 15, 20, 25, 30],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500]})

In [ ]:
gbrt_best = GradientBoostingRegressor(random_state=0, max_depth=search_gbrt.best_params_['max_depth'],
                                      max_features=search_gbrt.best_params_['max_features'],
                                      n_estimators=search_gbrt.best_params_['n_estimators'])
gbrt_best.fit(train_x, train_y)

print("train set R^2 : {:.3f}".format(gbrt_best.score(train_x, train_y)))
print("valid set R^2 : {:.3f}".format(gbrt_best.score(valid_x, valid_y)))

train set R^2 : 0.999
valid set R^2 : 0.387


## **k neighbors 회귀**

In [ ]:
param_knn={'n_neighbors':range(1,500)}

search_knn = RandomizedSearchCV(estimator=KNeighborsRegressor(), cv=5, param_distributions=param_knn, n_jobs=-1, n_iter=100)
search_knn.fit(train_x, train_y)

RandomizedSearchCV(cv=5, estimator=KNeighborsRegressor(), n_iter=100, n_jobs=-1,
                   param_distributions={'n_neighbors': range(1, 500)})

In [ ]:
kn_best = KNeighborsRegressor(n_neighbors=search_knn.best_params_['n_neighbors'])
kn_best.fit(train_x, train_y)

print("train set R^2 : {:.3f}".format(kn_best.score(train_x, train_y)))
print("valid set R^2 : {:.3f}".format(kn_best.score(valid_x, valid_y)))

train set R^2 : 1.000
valid set R^2 : 0.930


# voting regression

- 아무래도 train set과 validation set 성능의 간극을 좁힐 수 없어  
- 가장 성능이 좋게 나온 knn을 base estimator로 하여 세 앙상블 모델(pasting, bagging, boosting)을 만듦

아래 코드는 넣을지 말지 고민중,, 메타 러너로 랜포 들어간 게 설명 불가

In [ ]:
# 그 전에 두개의 부스팅 모델을 기반으로 스태킹도 시도해보았지만, 성능이 좋지 않아 앙상블 모델은 저렇게 세개만 사용하기로 함

GBC = GradientBoostingRegressor()
gb_param_grid = {'n_estimators' : [100,200,300,400,500],'learning_rate': [0.1, 0.05, 0.01],'max_depth': [2,4,6], 'max_features': [0.3, 0.1,0.5,0.9] }
gsGBC = GridSearchCV(GBC, param_grid = gb_param_grid, cv=5, scoring="neg_mean_squared_error", n_jobs=-1)
gsGBC.fit(train_x, train_y)
GBC_best = gsGBC.best_estimator_

XGB = XGBRegressor()
xgb_param_grid = {'learning_rate': [1,0.1,0.01,0.001],'n_estimators': [50, 100, 200, 500], 'max_depth' : [2,4,6]}
gsXGB = GridSearchCV(XGB, param_grid = xgb_param_grid, cv=5, scoring="neg_mean_squared_error", n_jobs=-1)
gsXGB.fit(train_x, train_y)
XGB_best = gsXGB.best_estimator_

st_re = StackingRegressor(regressors=[XGB_best, GBC_best], meta_regressor=RandomForestRegressor())
st_re.score(train_x, train_y)

## base estimator : knn regression

In [ ]:
kn_reg = KNeighborsRegressor(n_neighbors=6)

## train + valid (더이상 성능 확인 필요x)

In [ ]:
train = df.loc[df.기준년월!=202005, ['아파트세대수', '아파트동수', '아파트면적값','면적', '인구', '세대', '대형마트수_8km이내',
                                 '지하철수_2km이내', '학교수_1km이내', '공원수_2.6km이내', '인구밀도', '세대당주차대수', '아파트준공일자',
                                 'CCTV수','부대시설갯수', '승강기수', '관리사무소', '노인정', '문고', '보육시설', '어린이놀이터', '유치원',
                                 '자전거보관소', '주민공동시설', '커뮤니티공간', '휴게시설', '주택거래건수', 
                                 'mean.주가수익비율', 'mean.주가현금흐름비율', 'moving_num','y']]
test = df.loc[df.기준년월==202005, ['아파트세대수', '아파트동수', '아파트면적값','면적', '인구', '세대', '대형마트수_8km이내',
                                 '지하철수_2km이내', '학교수_1km이내', '공원수_2.6km이내', '인구밀도', '세대당주차대수', '아파트준공일자',
                                 'CCTV수','부대시설갯수', '승강기수', '관리사무소', '노인정', '문고', '보육시설', '어린이놀이터', '유치원',
                                 '자전거보관소', '주민공동시설', '커뮤니티공간', '휴게시설', '주택거래건수', 
                                 'mean.주가수익비율', 'mean.주가현금흐름비율', 'moving_num','y']]

train_x, train_y = get_xy(train)
test_x, test_y = get_xy(test)

## voting regressor

In [ ]:
pas_reg = BaggingRegressor(base_estimator = kn_reg, bootstrap=False, random_state=0)
bag_reg = BaggingRegressor(base_estimator = kn_reg, bootstrap=True, random_state=0)
ada_reg = AdaBoostRegressor(base_estimator = kn_reg, random_state=0)

param_ensemble = {'n_estimators':[100,200,300,400,500], 'max_features':[5,10,15,20,25,30]} 
param_adaboost = {'n_estimators':[100,200,300,400,500], 'learning_rate':[0.001,0.01,0.1,0.5,0.9]} 

def random_cv(obj, grid) :
    search_method = RandomizedSearchCV(estimator=obj, cv=5, param_distributions=grid, n_jobs=-1)
    search_method.fit(train_x, train_y)
    best = search_method.best_params_
    return best

pas_best = random_cv(pas_reg, param_ensemble)
pas_best = BaggingRegressor(base_estimator = kn_reg, bootstrap=False, random_state=0, n_estimators=pas_best['n_estimators'], max_features=pas_best['max_features'])

bag_best = random_cv(bag_reg, param_ensemble)
bag_best = BaggingRegressor(base_estimator = kn_reg, bootstrap=True, random_state=0, n_estimators=bag_best['n_estimators'], max_features=bag_best['max_features'])

ada_best = random_cv(ada_reg, param_adaboost)
ada_best = AdaBoostRegressor(base_estimator = kn_reg, random_state=0, n_estimators=ada_best['n_estimators'], learning_rate=ada_best['learning_rate'])

In [ ]:
from sklearn.ensemble import VotingRegressor

In [ ]:
pas_best = BaggingRegressor(base_estimator = kn_reg, bootstrap=False, random_state=0, n_estimators=300, max_features=25)
bag_best = BaggingRegressor(base_estimator = kn_reg, bootstrap=True, random_state=0, n_estimators=300, max_features=25)
ada_best = AdaBoostRegressor(base_estimator = kn_reg, random_state=0, n_estimators=400, learning_rate=0.1)

In [ ]:
voting_reg = VotingRegressor(
    estimators=[('bag', bag_best), ('pas', pas_best),('ada', ada_best)])

voting_reg.fit(train_x, train_y)

VotingRegressor(estimators=[('bag',
                             BaggingRegressor(base_estimator=KNeighborsRegressor(n_neighbors=6),
                                              max_features=25, n_estimators=300,
                                              random_state=0)),
                            ('pas',
                             BaggingRegressor(base_estimator=KNeighborsRegressor(n_neighbors=6),
                                              bootstrap=False, max_features=25,
                                              n_estimators=300,
                                              random_state=0)),
                            ('ada',
                             AdaBoostRegressor(base_estimator=KNeighborsRegressor(n_neighbors=6),
                                               learning_rate=0.1,
                                               n_estimators=400,
                                               random_state=0))])

# check and eliminate outlier obs. from residual

In [ ]:
train['pred'] = voting_reg.predict(train_x)
train['resd'] = np.abs(train.y - train.pred)

In [ ]:
train.resd.describe()

count    33312.000000
mean         0.038594
std          0.030438
min          0.000005
25%          0.015428
50%          0.032417
75%          0.054218
max          0.331815
Name: resd, dtype: float64

In [ ]:
train = train.loc[train.resd < 0.3, : ]

train_x = train.drop(['y','pred','resd'], axis=1)
train_y = train.y

In [ ]:
pas_best = random_cv(pas_reg)
pas_best = BaggingRegressor(base_estimator = kn_reg, bootstrap=False, random_state=0,
                           n_estimators=pas_best['n_estimators'], max_features=pas_best['max_features'])
bag_best = random_cv(bag_reg)
bag_best = BaggingRegressor(base_estimator = kn_reg, bootstrap=True, random_state=0,
                           n_estimators=bag_best['n_estimators'], max_features=bag_best['max_features'])
ada_best = random_cv(ada_reg)
ada_best = AdaBoostRegressor(base_estimator = kn_reg, random_state=0,
                           n_estimators=ada_best['n_estimators'], max_features=ada_best['max_features'])

voting_reg = VotingRegressor(
    estimators=[('bag', bag_best), ('pas', pas_best),('ada', ada_best)])

voting_reg.fit(train_x, train_y)

# final predict

residual 제거 X

In [ ]:
pred_y = voting_reg.predict(test_x)

## RMSE
np.sqrt(np.mean((np.exp(pred_y) - np.exp(test_y))**2))

517352.09505940485